In [1]:
# -*- coding: utf-8 -*-
import nltk
nltk.download('punkt')
import textprocess as tp
import numpy as np

[nltk_data] Downloading package punkt to /Users/ks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/wd/0n3639rd5_x4t0dx1_tqvyvw0000gn/T/jieba.cache


加載用戶詞典 ......


Loading model cost 0.677 seconds.
Prefix dict has been built successfully.


In [2]:
posdict = tp.read_lines("data/pos.txt")
negdict = tp.read_lines("data/neg.txt")

In [3]:
# 程度副詞詞典
mostdict = tp.read_lines('data/most.txt')   # 權值為2
verydict = tp.read_lines('data/very.txt')   # 權值為1.5
moredict = tp.read_lines('data/more.txt')   # 權值為1.25
ishdict = tp.read_lines('data/ish.txt')   # 權值為0.5
insufficientdict = tp.read_lines('data/insufficiently.txt')  # 权權值為0.25
inversedict = tp.read_lines('data/inverse.txt')  # 權值為-1

In [4]:
def match(word, sentiment_value):
    if word in mostdict:
        sentiment_value *= 2.0
    elif word in verydict:
        sentiment_value *= 1.75
    elif word in moredict:
        sentiment_value *= 1.5
    elif word in ishdict:
        sentiment_value *= 1.2
    elif word in insufficientdict:
        sentiment_value *= 0.5
    elif word in inversedict:
        #print "inversedict", word
        sentiment_value *= -1
    return sentiment_value

In [5]:
def transform_to_positive_num(poscount, negcount):
    pos_count = 0
    neg_count = 0
    if poscount < 0 and negcount >= 0:
        neg_count += negcount - poscount
        pos_count = 0
    elif negcount < 0 and poscount >= 0:
        pos_count = poscount - negcount
        neg_count = 0
    elif poscount < 0 and negcount < 0:
        neg_count = -poscount
        pos_count = -negcount
    else:
        pos_count = poscount
        neg_count = negcount
    return (pos_count, neg_count)

In [13]:
def single_review_sentiment_score(inputtext):
    single_review_senti_score = []
    cuted_review = tp.cut_sentence(inputtext)  # 句子切分，單獨對每個句子進行分析

    for sent in cuted_review:
        seg_sent = tp.segmentation(sent)   # 分詞
        seg_sent = tp.del_stopwords(seg_sent)[:]
        #for w in seg_sent:
        #   print (w,)
        
        i = 0    # 記錄掃描到的詞的位置
        s = 0    # 記錄情感詞的位置
        poscount = 0    # 記錄該分句中的積極情感得分
        negcount = 0    # 記錄該分句中的消極情感得分

        for word in seg_sent:   # 逐詞分析
            print (word)
            if word in posdict:  # 如果是積極情感詞
                #print "posword:", word
                poscount += 1   # 積極得分+1
                for w in seg_sent[s:i]:
                    poscount = match(w, poscount)
                #print "poscount:", poscount
                s = i + 1  # 記錄情感詞的位置變化

            elif word in negdict:  # 如果是消極情感詞
                #print "negword:", word
                negcount += 1
                for w in seg_sent[s:i]:
                    negcount = match(w, negcount)
                #print "negcount:", negcount
                s = i + 1

            # 如果是感嘆號，表示已經到本句句尾
            elif word == "！" or word == "!":
                for w2 in seg_sent[::-1]:  # 倒序掃描感嘆號前的情感詞，發現後權值+2，然後退出循環
                    if w2 in posdict:
                        poscount += 2
                        break
                    elif w2 in negdict:
                        negcount += 2
                        break
            i += 1
        #print "poscount,negcount", poscount, negcount
        single_review_senti_score.append(transform_to_positive_num(poscount, negcount))   # 對得分做最後處理
    pos_result, neg_result = 0, 0   # 分別記錄積極情感總得分和消極情感總得分
    for res1, res2 in single_review_senti_score:  # 每個分句循環累加
        pos_result += res1
        neg_result += res2
    #print (pos_result, neg_result)
    result = pos_result - neg_result
    result = round(result, 1)
    return result

In [14]:
def getscore(text):
    score = single_review_sentiment_score(text)
    
    return score

In [23]:
if __name__ == '__main__':
    text = input()

    if (text != ""):
        score = getscore(text)

        if (score < 0):
            print("Dissatisfaction")
        elif (score == 0):
            print("Normal")
        elif (score > 0):
            print("Satisfaction")